## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-24-08-24-20 +0000,nyt,China’s Tropical Resort Island Orders Evacuati...,https://www.nytimes.com/2025/08/24/world/asia/...
1,2025-08-24-08-17-41 +0000,bbc,Pub and travel bans proposed under sentencing ...,https://www.bbc.com/news/articles/c5ypej14j2xo...
2,2025-08-24-07-48-42 +0000,bbc,Russia blames nuclear site attack on Ukraine a...,https://www.bbc.com/news/articles/czxy2v9dzgxo...
3,2025-08-24-07-36-33 +0000,bbc,My trip to North Korea's 'Benidorm' - flanked ...,https://www.bbc.com/news/articles/c707d1ez0kno...
4,2025-08-24-07-27-37 +0000,nypost,Tropical Storm Fernand forms southeast of Bermuda,https://nypost.com/2025/08/24/us-news/tropical...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2343/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
55,trump,25
21,ukraine,8
80,report,6
16,russia,6
234,new,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
104,2025-08-23-13-00-00 +0000,wsj,When President Trump called Gavin Newsom “News...,https://www.wsj.com/politics/policy/to-fight-t...,65
110,2025-08-23-12-29-00 +0000,wsj,The Texas Senate approved a redistricting plan...,https://www.wsj.com/politics/policy/texas-cong...,55
7,2025-08-24-07-02-31 +0000,nypost,Illinois Gov. Pritzker says Trump trying to ‘m...,https://nypost.com/2025/08/24/us-news/illinois...,44
17,2025-08-24-04-12-28 +0000,bbc,Trump's plan to send troops to Chicago is abus...,https://www.bbc.com/news/articles/cd7yperjxneo...,42
19,2025-08-24-04-01-16 +0000,nyt,China Looms Large as South Korea’s President M...,https://www.nytimes.com/2025/08/24/world/asia/...,41


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
104,65,2025-08-23-13-00-00 +0000,wsj,When President Trump called Gavin Newsom “News...,https://www.wsj.com/politics/policy/to-fight-t...
2,29,2025-08-24-07-48-42 +0000,bbc,Russia blames nuclear site attack on Ukraine a...,https://www.bbc.com/news/articles/czxy2v9dzgxo...
30,28,2025-08-24-02-00-00 +0000,wsj,Bashing the president’s tariff policies could ...,https://www.wsj.com/politics/policy/gop-tariff...
34,27,2025-08-24-01-14-02 +0000,nypost,"Rikers Island inmate found dead in cell, marki...",https://nypost.com/2025/08/23/us-news/rikers-i...
108,25,2025-08-23-12-42-42 +0000,cbc,U.S. seeks to deport Kilmar Abrego Garcia to U...,https://www.cbc.ca/news/world/kilmar-abrego-ga...
82,24,2025-08-23-18-30-32 +0000,nypost,"North Korea builds secret military base, incre...",https://nypost.com/2025/08/23/world-news/north...
43,24,2025-08-24-00-05-20 +0000,nypost,Zohran Mamdani fails at bench press while tryi...,https://nypost.com/2025/08/23/us-news/zohran-m...
75,22,2025-08-23-19-36-55 +0000,nypost,FBI believes Bolton should face charges over a...,https://nypost.com/2025/08/23/us-news/fbi-beli...
70,22,2025-08-23-20-15-36 +0000,nypost,"Putin war hero, the ‘Executioner,’ exposed as ...",https://nypost.com/2025/08/23/world-news/putin...
15,20,2025-08-24-04-53-54 +0000,nypost,Rhode Island prosecutor who told cops they’d ‘...,https://nypost.com/2025/08/24/us-news/rhode-is...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
